In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer as wnl
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

sys.path.append('./data')
sys.path.append('./models')

# from train_classifier import load_data

# Downloads for NLTK tools
nltk.download('words', quiet=True);
nltk.download('wordnet', quiet=True);
nltk.download('punkt_tab', quiet=True);
nltk.download('stopwords', quiet=True)


# Project Overview
## ETL Pipeline: process_data.py
In a Python script, process_data.py, write a data cleaning pipeline that:
- Loads the messages and categories datasets
- Merges the two datasets
- Cleans the data
- Stores it in a SQLite database

## ML pipeline: train_classifier.py
In a Python script, train_classifier.py, write a machine learning pipeline that:
- Loads data from the SQLite database
- Splits the dataset into training and test sets
- Builds a text processing and machine learning pipeline
- Trains and tunes a model using GridSearchCV
- Outputs results on the test set
- Exports the final model as a pickle file

## Flask Web App
We are providing much of the flask web app for you, but feel free to add extra features depending on your knowledge of flask, html, css and javascript. For this part, you'll need to:
- Modify file paths for database and model as needed
- Add data visualizations using Plotly in the web app. One example is provided for you.

## Github and Code Quality
Your project will also be graded based on the following:
- Use of Git and Github
- Strong documentation
- Clean and modular code
- Follow the [RUBRIC](https://learn.udacity.com/nanodegrees/nd025/parts/cd0018/lessons/e692c8ed-b713-464b-95ac-72d93a35b4fc/concepts/e692c8ed-b713-464b-95ac-72d93a35b4fc-project-rubric) when you work on your project to assure you meet all of the necessary criteria for developing the pipelines and web app.

In [ ]:
DATA_DIR = './data'
database_filepath = os.path.join(DATA_DIR, 'DisasterResponse.db')
messages_filepath = os.path.join(DATA_DIR, 'disaster_messages.csv')
categories_filepath = os.path.join(DATA_DIR, 'disaster_categories.csv')

# debug process_data.py

In [ ]:
from process_data import clean_data, load_data, save_data, load_categories_data, load_messages_data

In [ ]:
# m = load_messages_data(messages_filepath)
# c = load_categories_data(categories_filepath)
# df = m.merge(c, on='id')
# df = pd.get_dummies(df, columns=['related', 'genre'], drop_first=True)

In [ ]:
df = clean_data(load_data(messages_filepath, categories_filepath))
save_data(df, database_filepath)

# debug train_classifier.py

In [ ]:
from train_classifier import load_data, display_results

# train_classifier.py

In [ ]:
DATA_DIR = './data'
database_filepath = os.path.join(DATA_DIR, 'DisasterResponse.db')
X, Y, category_names = load_data(database_filepath)

# create dummies for genre and related - we can "recover" them later
Y_dummies = ['related']
X_dummies = ['genre']
Y = pd.get_dummies(Y, columns=Y_dummies, dtype=int)
X = pd.get_dummies(X, columns=X_dummies, dtype=int)

X = X[['message']]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=2/3, test_size=1/3, random_state=42)


In [ ]:
def recover_dummies(df, dummies_in_df):
    dummy_columns = []
    for dummy in dummies_in_df:
        dummy_columns += [i for i in df.columns if f'{dummy}_' in i]
        df = pd.concat([
            df.drop(dummy_columns, axis=1),
            pd.from_dummies(df[dummy_columns], sep='_'),
        ],axis=1)
    return df

# X = recover_dummies(X, X_dummies)
# Y = recover_dummies(Y, Y_dummies)


The script uses a custom tokenize function using nltk to case normalize, lemmatize, and tokenize text. This function is used in the machine learning pipeline to vectorize and then apply TF-IDF to the text.

In [ ]:
def tokenize(text, lemmatize=True, make_lowercase=True, remove_non_words=True, remove_stop_words=True, verbose=False):
    def get_tokens_per_row(text, make_lowercase):
        if verbose:
            print('... tokenizing and converting to lowercase.' if make_lowercase else '... tokenizing.')
        return [nltk.word_tokenize(s.lower() if make_lowercase else s) for s in text]
    
    def _remove_non_words(tokens_by_message):
        if verbose:
            print('... removing non-words.')
        answer = []
        for tokens in tokens_by_message:
            answer.append([i for i in tokens if i.isalpha()])
        return answer

    def _remove_stop_words(tokens_by_message, stop_words=set(nltk.corpus.stopwords.words('english'))):
        if verbose:
            print('... removing stop words.')
        answer = []
        for tokens in tokens_by_message:
            answer.append([i for  i in tokens if i not in stop_words])
        return answer
    
    def _lemmatize(tokens_by_message, lemmatizer=nltk.stem.WordNetLemmatizer()):
        if verbose:
            print('... lemmatizing.')
        answer = []
        for tokens in tokens_by_message:
            answer.append([lemmatizer.lemmatize(i).strip() for i in tokens])
        return answer

    tokens_by_message = get_tokens_per_row(text, make_lowercase)

    if lemmatize:
        tokens_by_message = _lemmatize(tokens_by_message)

    if remove_stop_words:
        tokens_by_message = _remove_stop_words(tokens_by_message)
        
    if remove_non_words:
        tokens_by_message = _remove_non_words(tokens_by_message)

    # now combine back to strings (one per row) so we can pass it back to TfidfTransformer
    processed_messages = [' '.join(tokens) for tokens in tokens_by_message]

    return processed_messages

In [ ]:
# text = X.message.iloc[0:50]
# processed_messages = tokenize(text)
# vectorizer = TfidfVectorizer() 
# tfidf = vectorizer.fit_transform(processed_messages)

In [ ]:
# class Tokenizer(BaseEstimator, TransformerMixin):
#     def __init__(self, column_to_process='message'):
#         self.column_to_process = column_to_process

#     def fit(self, X, y=None):
#         return self
#     def transform(self, X, y=None):
#         return tokenize(X[self.column_to_process])


In [ ]:
# Thank you: https://stackoverflow.com/questions/67768470/valueerror-found-input-variables-with-inconsistent-numbers-of-samples-6-80

transformer = ColumnTransformer([
    ('vect', TfidfVectorizer(tokenizer=tokenize), 'message')
], remainder='passthrough')

pipeline = Pipeline([
    ('vect', transformer),
    ('model', RandomForestClassifier()),
])

In [ ]:
pipeline.fit(X_train, Y_train)

In [ ]:
Y_pred = pipeline.predict(X_test)

In [ ]:
display_results(Y_test, Y_pred)

# app.py

# CRISP-DM Flow
- ALL of this should go in the README.md file at the end of the project.
- The rubric states the README should contain a lot of info.

## Business Understanding
- The purpose of this project is to identify an efficient way to model emergency messages to determine which are most informative for making a quick disaster response decision. This is important as resources for disaster response are limited and false positives can prevent those resources from being deployed where they are needed most.

In [ ]:
# int_columns = [k for k, v in df.dtypes.items() if v in [int, np.int64]]
# df[[k for k, v in df.dtypes.items() if v in [int, np.int64]]].describe().T


## Data Understanding
- The dataset is raw, containing an identifier along with communication information that can contain an English 'message', a non-English 'original' messge, and a 'genre' (type of communication). This insformation is provided in CSV form.
- The content of each row is inconsistent - for example, some of the lines contain more than 4 commas (which would cleanly correspond to the 4 expected fields), some contain consecutive commas (where one of the messages is missing), and some contain quotation marks separating the English and non-English messages.
- There is also categorical information stored in a separate CSV, which provides some informative identifiers that are pre-associated with each message (and link by an 'id' field in each set of data).
    - This CSV data is further subdivided into key/value pairs separated by a semicolon, e.g. 'id,key1-value1;key2-value2; ...' etc.
    - There are 36 categeries in the file, and the rubric says these are to be used as the ***responses*** for the multi-output classification task.
    - Note that the values in the category values are integers - mostly boolean (0 or 1) except for **related** which is ternery (0/1/2).
        - If using a decision tree style classifier, we do not need to create dummies for the **related** variable.
    - These categories are also NOT mutually exclusive (the cross-sectional sum of them can be greater than 1).

- **load_data.py** loads the raw message and category information, first splits out the 'id' and the 'categories' key/value pairs, then splits the key/value pairs into columns of data with integer data, and joins each dataset (on 'id') and passes that joined dataframe along for cleaning in the next phase.

## Data Preparation
- **clean_data.py** attempts to separate the Enlish and non-English components of the raw nessage, and stores the processed English-only message as a new field in the data, allowing it to be more easily processed in the machine learning pipeline.
- It also converts the 'genre' field into a set of dummies with boolean values (n-1 categories).

## Data Modeling

## Result Evaluation

## Deployment